### Database with 4*4 images

shapes : square, triangle, cirle
colors : light red, light blue, light yellow, light green

dataset size : 1,000,000

probability to have a symbol at each position : 0.5

Same as v0.0.1 but paths are relative to the db root and not absolute



In [4]:
db_dir = "/home/jleguy/Documents/postdoc/data/PatImgXAI_data/db0.1.2/"

In [5]:
from xaipatimg.datagen.dbimg import load_db

db = load_db(db_dir)

In [4]:
# Number of images generated
NBGEN = 1000000

# Grid division of each image
X_DIVISIONS = 4
Y_DIVISIONS = 4

# Size of the images in pixels
img_size = (700, 700)

# Probability to generate a geometrical shape at each position in the grid
SHAPE_PROB = 0.5

# Define available shapes
SHAPES = ['circle', 'square', 'triangle']
COLORS  = ["#F86C62", "#7AB0CD", "#F4D67B", "#87C09C"]

In [4]:
import numpy as np
from xaipatimg.datagen.dbimg import generate_uuid
import os
import tqdm

for genidx in tqdm.tqdm(range(NBGEN)):
    content = []
    for i in range(X_DIVISIONS):
        for j in range(Y_DIVISIONS):
            if np.random.random() < SHAPE_PROB:
                content.append({
                    "shape": np.random.choice(SHAPES),
                    "pos": (i, j),
                    "color": np.random.choice(COLORS)
                })

    imgid = generate_uuid()
    db[imgid] = {
        "path": os.path.join("img", imgid + ".png"),
        "division" : (X_DIVISIONS, Y_DIVISIONS),
        "size": img_size,
        "content": content
    }

100%|██████████| 1000000/1000000 [01:41<00:00, 9814.20it/s]


In [5]:
from xaipatimg.datagen.genimg import gen_img_and_save_db
gen_img_and_save_db(db, db_dir, overwrite=False, n_jobs=36)

100%|██████████| 1000000/1000000 [17:24<00:00, 957.47it/s]


### Create dataset with blue diagonal rule

In [4]:
def blue_diagonal_rule(img_content):
    symbols_in_diagonal = 0
    for c in img_content:
        if c["pos"][0] == c["pos"][1]:
            symbols_in_diagonal += 1
            if c["color"] != "#7AB0CD":
                return False
    return symbols_in_diagonal >= 1

In [5]:
from xaipatimg.datagen.gendataset import create_dataset_based_on_rule
import os
csv_name_train = "bluediagonal_train.csv"
csv_name_test = "bluediagonal_test.csv"
csv_name_valid = "bluediagonal_valid.csv"
sample_img_path = os.path.join(db_dir, "bluediagonal_train")
create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_valid, csv_name_test, test_size=2000, valid_size=0.2,
                             dataset_pos_samples_nb=11000, dataset_neg_samples_nb=11000, rule_fun=blue_diagonal_rule)

100%|██████████| 1000000/1000000 [00:01<00:00, 971890.67it/s]


In [6]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, sample_img_path, 1000, 1000)

16001it [00:00, 253035.30it/s]


### Create dataset with exactly four squares rule

In [4]:
def four_squares_total(img_content):
    sq_total = 0
    for c in img_content:
        if c["shape"] == "square":
            sq_total += 1
    return sq_total == 4

In [5]:
from xaipatimg.datagen.gendataset import create_dataset_based_on_rule
import os
csv_name_train = "foursquares_train.csv"
csv_name_test = "foursquares_test.csv"
csv_name_valid = "foursquares_valid.csv"
sample_img_path = os.path.join(db_dir, "foursquares_train")

create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_valid, csv_name_test, test_size=2000, valid_size=0.2,
                             dataset_pos_samples_nb=11000, dataset_neg_samples_nb=11000, rule_fun=four_squares_total)

100%|██████████| 1000000/1000000 [00:05<00:00, 168775.14it/s]


In [6]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, output_dir_path=sample_img_path, pos_samples_nb=1000, neg_samples_nb=1000)

16001it [00:00, 16030.99it/s]


### Create dataset with exactly three squares rule


In [ ]:
def three_squares_total(img_content):
    sq_total = 0
    for c in img_content:
        if c["shape"] == "square":
            sq_total += 1
    return sq_total == 3

In [ ]:
csv_name_train = "threesquares_train.csv"
csv_name_test = "threesquares_test.csv"
csv_name_valid = "threesquares_valid.csv"
sample_img_path = os.path.join(db_dir, "threesquares_train")

create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_valid, csv_name_test, test_size=2000, valid_size=0.2,
                             dataset_pos_samples_nb=11000, dataset_neg_samples_nb=11000, rule_fun=three_squares_total)

In [ ]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, output_dir_path=sample_img_path, pos_samples_nb=1000, neg_samples_nb=1000)

### Create dataset with any image rule (to show sample of the full database)

In [6]:
def any_image(img_content):
    return True

from xaipatimg.datagen.gendataset import create_dataset_based_on_rule
import os
csv_name_train = "any_train.csv"
csv_name_test = "any_test.csv"
csv_name_valid = "any_valid.csv"
sample_img_path = os.path.join(db_dir, "any_sample")

create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_valid, csv_name_test, valid_size=0.2, test_size=2000,
                             dataset_pos_samples_nb=1000, dataset_neg_samples_nb=0, rule_fun=any_image)

100%|██████████| 1000000/1000000 [00:00<00:00, 2549121.43it/s]


In [7]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, output_dir_path=sample_img_path, pos_samples_nb=1000, neg_samples_nb=0)

801it [00:00, 29209.41it/s]


### Create dataset "twice as many red symbols as blue symbols"

In [11]:
def twice_as_many_red_as_blue(img_content):
    blue_total = 0
    red_total = 0

    for c in img_content:
        if c["color"] == "#F86C62":
            red_total += 1
        elif c["color"] == "#7AB0CD":
            blue_total += 1

    return red_total == 2 * blue_total

In [12]:
import os
from xaipatimg.datagen.gendataset import create_dataset_based_on_rule

csv_name_train = "twiceasmanyredasblue_train.csv"
csv_name_test = "twiceasmanyredasblue_test.csv"
csv_name_valid = "twiceasmanyredasblue_valid.csv"
sample_img_path = os.path.join(db_dir, "twiceasmanyredasblue_train")

create_dataset_based_on_rule(db_dir, csv_name_train, csv_name_valid, csv_name_test, test_size=2000, valid_size=0.2,
                             dataset_pos_samples_nb=11000, dataset_neg_samples_nb=11000, rule_fun=twice_as_many_red_as_blue)


100%|██████████| 1000000/1000000 [00:02<00:00, 375144.88it/s]


In [13]:
from xaipatimg.datagen.gendataset import extract_sample_from_dataset
extract_sample_from_dataset(db_dir, csv_name_train, output_dir_path=sample_img_path, pos_samples_nb=1000, neg_samples_nb=1000)

16001it [00:00, 30929.44it/s]


# Checking for duplicates in the generated DB

In [6]:
import tqdm

content_dict = {}
nb_duplicates = 0

for k, v in tqdm.tqdm(db.items()):
    if str(v["content"]) in content_dict:
        nb_duplicates += 1
    else:
        content_dict[str(v["content"])] = True

print(nb_duplicates)

100%|██████████| 1000000/1000000 [00:17<00:00, 58309.13it/s]

257


There was actually 257 duplicates so the DB is generated again (v0.1.3)